In [20]:
import tempfile
import os

# Set temp to E drive
custom_temp = "E:/temp"
os.makedirs(custom_temp, exist_ok=True)
tempfile.tempdir = custom_temp
os.environ["TMP"] = custom_temp
os.environ["TEMP"] = custom_temp

print("✅ Temp folder set to:", tempfile.gettempdir())


✅ Temp folder set to: E:/temp


In [21]:
import shutil
free = shutil.disk_usage(tempfile.gettempdir()).free
print(f"📂 Free space in temp: {free // (1024 ** 2)} MB")


📂 Free space in temp: 49123 MB


In [22]:
!pip install gradio openai yt_dlp ffmpeg-python requests
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to e:\temp\pip-req-build-qqa24ov_
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'E:\temp\pip-req-build-qqa24ov_'


In [23]:
import gradio as gr
import tempfile
import os
import subprocess
import whisper
import requests
import yt_dlp
import textwrap
import getpass
from openai import OpenAI
import cv2
from PIL import Image
import numpy as np
import base64
import io


In [24]:
import getpass
from openai import OpenAI  # Ensure correct import based on your OpenAI library version

# Securely prompt user for their OpenRouter API key
api_key = getpass.getpass("🔒 Please enter your OpenRouter API key (input hidden): ")

# Initialize the OpenAI-compatible client for OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key
)

🔒 Please enter your OpenRouter API key (input hidden):  ········


In [25]:
whisper_model = whisper.load_model("base")


In [26]:
def download_youtube_video(youtube_url):
    temp_dir = tempfile.mkdtemp()
    output_path = os.path.join(temp_dir, "video.%(ext)s")

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'merge_output_format': 'mp4',
        'ffmpeg_location': r'C:\Users\AFI\Downloads\ffmpeg-7.0.2-essentials_build\bin\ffmpeg.exe',
        'outtmpl': output_path,
        'quiet': True,
        'noplaylist': True
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    downloaded_file = output_path.replace("%(ext)s", "mp4")
    if not os.path.exists(downloaded_file):
        raise FileNotFoundError("Video download failed.")
    return downloaded_file


def download_direct_video(url):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        temp_video = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
        for chunk in response.iter_content(chunk_size=1024):
            temp_video.write(chunk)
        temp_video.close()
        return temp_video.name
    else:
        raise ValueError("Failed to download video from URL.")


def is_valid_video(video_path):
    try:
        result = subprocess.run(["ffmpeg", "-v", "error", "-i", video_path, "-f", "null", "-"],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return result.returncode == 0
    except Exception:
        return False


In [31]:
def transcribe_video(video_path):
    result = whisper_model.transcribe(video_path)
    return result["text"]

def extract_key_frames(video_path, num_frames=1, max_dim=256):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    selected_frames = []
    if total_frames == 0:
        return []
    step = max(total_frames // (num_frames + 1), 1)

    for i in range(1, num_frames + 1):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        success, frame = cap.read()
        if success:
            h, w = frame.shape[:2]
            scale = max_dim / max(h, w)
            resized = cv2.resize(frame, (int(w * scale), int(h * scale)))
            _, buffer = cv2.imencode(".jpg", resized, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
            b64_image = base64.b64encode(buffer.tobytes()).decode('utf-8')
            selected_frames.append(f"data:image/jpeg;base64,{b64_image}")
    cap.release()
    return selected_frames

def describe_video_with_images(images_b64):
    descriptions = []
    for img_b64 in images_b64:
        prompt = f"""
You are a professional scriptwriter for iboothme Creative, mimicking their exact YouTube description style.

The audio track had no usable speech, so you're analyzing the visual content.

Follow this structure exactly (no headings):
- First line: metadata view count and upload date, with hashtags if available.
- Next three lines:
  📩 info@iboothme.com  
  📞 +971 4 448 8563  
  👉🏼 https://www.iboothme.com

- Title line: product name + subtitle + hashtags.

- Greeting: "Hi everyone, Welcome to the iboothme Channel."

- A paragraph describing the product, tech, and guest experience (1–2 sentences).

- A paragraph with use cases or extra promotions.

- A line to subscribe or learn more.

- Optionally a link (Instagram/location/brochure) if relevant.

- At end: 3–7 related hashtags or keyphrases.

Image for reference (base64 JPEG):
\"\"\"{img_b64}\"\"\"

Now generate the full description exactly in this style. Do not repeat any parts.
"""
        resp = client.chat.completions.create(
            model="mistralai/mistral-7b-instruct:free",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.7
        )
        result = resp.choices[0].message.content.strip()

        # Remove accidental duplication
        first_half = result[:len(result)//2]
        if result.endswith(first_half):
            descriptions.append(first_half.strip())
        else:
            descriptions.append(result)

    return descriptions[0] if descriptions else ""



In [35]:
def clean_transcript(text):
    lines = text.splitlines()
    filtered = [line for line in lines if "iboothme.com" not in line and "+971" not in line]
    return "\n".join(filtered).strip()

def generate_with_openrouter(transcript_chunk):
    prompt = f"""
You are a professional scriptwriter for iboothme Creative, mimicking their exact YouTube description style.

Follow this structure exactly (no headings):
- First line: metadata view count and upload date, with hashtags if available.
- Next three lines:
  📩 info@iboothme.com  
  📞 +971 4 448 8563  
  👉🏼 https://www.iboothme.com

- Title line: product name + subtitle + hashtags.

- Greeting: "Hi everyone, Welcome to the iboothme Channel."

- A paragraph describing the product, tech, and guest experience (1–2 sentences).

- A paragraph with use cases or extra promotions.

- A line to subscribe or learn more.

- Optionally a link (Instagram/location/brochure) if relevant.

- At end: 3–7 related hashtags or keyphrases.

Transcript (for reference):
\"\"\"{transcript_chunk}\"\"\"

Now generate the full description exactly in this style.
"""
    resp = client.chat.completions.create(
        model="mistralai/mistral-7b-instruct:free",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.7
    )
    return resp.choices[0].message.content.strip()


In [36]:
def generate_description(input_mode, video_file, video_url):
    try:
        if input_mode == "Upload":
            if not video_file:
                return "Please upload a video file."
            if isinstance(video_file, str) and os.path.exists(video_file):
                video_path = video_file
            else:
                with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp:
                    temp.write(video_file.read())
                    video_path = temp.name
        elif input_mode == "URL":
            if not video_url:
                return "Please enter a video URL."
            if "youtube.com" in video_url or "youtu.be" in video_url:
                video_path = download_youtube_video(video_url)
            else:
                video_path = download_direct_video(video_url)
        else:
            return "Invalid input mode."

        if not is_valid_video(video_path):
            return "Error: The uploaded or downloaded video is not valid."

        transcript = transcribe_video(video_path).strip()

        if transcript:
            chunks = textwrap.wrap(transcript, 1500)
            result_parts = [generate_with_openrouter(chunk) for chunk in chunks[:1]]
            result = "\n\n".join(result_parts)
        else:
            frames_b64 = extract_key_frames(video_path)
            if not frames_b64:
                result = "Error: No usable audio or video content found."
            else:
                result = describe_video_with_images(frames_b64)

        if os.path.exists(video_path):
            os.remove(video_path)

        return result

    except Exception as e:
        return f"Error: {str(e)}"


In [37]:
with gr.Blocks() as interface:
    gr.Markdown("## 🎥 iboothme Creative – Description Generator")
    gr.Markdown("Upload a video or paste a YouTube URL (metadata pulled automatically).")

    input_mode = gr.Radio(["Upload", "URL"], value="Upload")
    video_file = gr.Video(visible=True)
    video_url = gr.Textbox(label="Video URL", visible=False)

    output_text = gr.Textbox(label="Generated Description", lines=12)

    def toggle(mode):
        return gr.update(visible=(mode=="Upload")), gr.update(visible=(mode=="URL"))

    input_mode.change(toggle, input_mode, outputs=(video_file, video_url))

    gr.Button("Generate").click(
        fn=generate_description,
        inputs=[input_mode, video_file, video_url],
        outputs=output_text
    )

interface.launch(share=True, inline=False)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://99a370b0add0c60d7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\AFI\anaconda3\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


C:\Users\AFI\anaconda3\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


C:\Users\AFI\anaconda3\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


C:\Users\AFI\anaconda3\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
